In [0]:
-- As a discontinued profile definition was taken
-- profile which had not any order in day 2  or day 3
-- As a day 2 or day 3 was taken business definition 
-- from totl amount sum/commulative becasue no any 
-- other definition was mentioned
SELECT pd_2_3.until_day, cb.* FROM customer_bronze cb
INNER JOIN profiles_data_days_2_3 pd_2_3
ON cb.customer_nk_num = pd_2_3.customer_nk_num AND 
cb.customer_version = pd_2_3.customer_version
WHERE pd_2_3.nk_num_order IS NULL 
ORDER BY pd_2_3.until_day, cb.customer_nk_num, cb.customer_version ASC